In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
data.head()

In [ ]:
df=data.copy()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['cp']=df['cp'].astype('object').replace({0: 'typical angina Value',
                                                  1:'atypical angina Value',
                                                  2: 'non-anginal pain Value',
                                                  3: 'asymptomatic trtbps' })

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
for i in df.drop('cp',axis=1).columns:
    plt.figure(figsize=(10,3))
    plt.subplot(1,2,1)
    sns.boxplot(df[i])
    plt.subplot(1,2,2)
    sns.distplot(df[i])
    plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(df['age'].sort_values())

In [ ]:
df=pd.get_dummies(df)
df.head()

In [ ]:
x=df.drop('output',axis=1)
y=df['output']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
x_scaled=StandardScaler().fit_transform(x)
X=pd.DataFrame(x_scaled,columns=x.columns)
X.head()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print('x_train shape:',x_train.shape)
print('x_test shape:',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

In [ ]:
import statsmodels.api as sm

In [ ]:
log_model=sm.Logit(y_train,sm.add_constant(x_train)).fit()
print(log_model.summary())

In [ ]:
log_model.pvalues>0.5

age and fbs have to remove.

In [ ]:
y_pred_prob=log_model.predict(sm.add_constant(x_test))
y_pred=[0 if x<0.5 else 1 for x in y_pred_prob]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('Accuracy of the model is:',round(accuracy_score(y_test,y_pred)*100,2),'.')

In [ ]:
# build model after of remove the pval test.
x=df.drop(['output','age','fbs'],axis=1)
y=df['output']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x_scaled=MinMaxScaler().fit_transform(x)
X=pd.DataFrame(x_scaled,columns=x.columns)
X.head()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
print('x_train shape:',x_train.shape)
print('x_test shape:',x_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
dt_model=DecisionTreeClassifier(criterion='gini',random_state=10).fit(x_train,y_train)

In [ ]:
plt.figure(figsize=(15,10))
tree.plot_tree(dt_model,class_names=['0','1'],filled=True)
plt.show()

In [ ]:
param_grid=[{'criterion':['gini','entropy'],'max_depth':range(1,30)}]

In [ ]:
grid_model=GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=param_grid,cv=3).fit(x_train,y_train)
grid_model.best_params_

In [ ]:
dt_model=DecisionTreeClassifier(criterion=grid_model.best_params_.get('criterion'),
                               max_depth=grid_model.best_params_.get('max_depth')).fit(x_train,y_train)
y_pred=dt_model.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
cm=confusion_matrix(y_test,y_pred)
conf_metrix=pd.DataFrame(cm,columns=['pred 1','pred 0'],index=['act 1','act 0'])
sns.heatmap(conf_metrix,annot=True)
plt.show()

In [ ]:
frp,trp,thes=roc_curve(y_test,dt_model.predict_proba(x_test)[:,1])
plt.figure(figsize=(15,8))
plt.plot(frp,trp)
plt.plot([0,1],[0,1],'r--')
plt.text(0.3,0.8,round(accuracy_score(y_test,y_pred)*100,4))
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.grid()
plt.show()

In [ ]:
#KNN model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=5,metric='euclidean').fit(x_train,y_train)

In [ ]:
y_pred=knn_model.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
#tuned parameter
pram_grid=[{'n_neighbors':range(1,15),'metric':['euclidean','minkowski','Hamming']}]
grid_model=GridSearchCV(estimator=KNeighborsClassifier(),param_grid=pram_grid,cv=5,scoring='r2').fit(x_train,y_train)
grid_model.best_params_


In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=grid_model.best_params_.get('n_neighbors'),
                               metric=grid_model.best_params_.get('metric')).fit(x_train,y_train)

In [ ]:
y_pred=knn_model.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
cm=confusion_matrix(y_test,y_pred)
conf_metrix=pd.DataFrame(cm,columns=['pred 1','pred 0'],index=['act 1','act 0'])
sns.heatmap(conf_metrix,annot=True)
plt.show()

In [ ]:
frp,trp,thes=roc_curve(y_test,knn_model.predict_proba(x_test)[:,1])
plt.figure(figsize=(15,8))
plt.plot(frp,trp)
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.grid()
plt.show()

In [ ]:
# Randomforest classification:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model=RandomForestClassifier(n_estimators=100,criterion='gini',random_state=10).fit(x_train,y_train)
y_pred=rf_model.predict(x_test)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
conf_matrix=pd.DataFrame(cm,columns=['pred 0','pred 1'],index=['act 1','act 0'])
sns.heatmap(conf_matrix,annot=True)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
frp,trp,thes=roc_curve(y_test,rf_model.predict_proba(x_test)[:,1])
plt.figure(figsize=(15,8))
plt.plot(frp,trp)
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.grid()
plt.show()

In [ ]:
rf_model.feature_importances_

In [ ]:
best=pd.DataFrame({'columns':x_train.columns,
                   'best feature':rf_model.feature_importances_}).sort_values(by='best feature',ascending=False).reset_index(drop=True)
best

In [ ]:
plt.figure(figsize=(15,8))
sns.barplot(best['columns'],best['best feature'])
plt.xticks(rotation=90)
plt.show()